CODE BLOCK: 1

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary



cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")


# Train data transformations
train_transforms = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(0, translate=(0.1, 0.1), scale = (0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

# Test data transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])


train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transforms)


batch_size = 128

test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle = False, num_workers=2 )
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True, num_workers=2)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Convolutions + BatchNorm
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.05)

        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(16)

        self.conv4 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn4   = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.05)

        self.conv5 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn5   = nn.BatchNorm2d(32)

        self.conv6 = nn.Conv2d(32, 32, kernel_size=1)
        self.bn6   = nn.BatchNorm2d(32)
        self.drop3 = nn.Dropout(0.05)

        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc  = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.drop3(x)

        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}


from tqdm import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc.append(100*correct/processed)
  train_losses.append(train_loss/len(train_loader))

def test(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


model = Net().to(device)
print(summary(model, input_size=(1, 28, 28)) )
print()
print()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Scheduler: reduce LR when val_loss plateaus
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

# New Line
criterion = nn.CrossEntropyLoss()
num_epochs = 20

for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train(model, device, train_loader, optimizer, criterion)
  test(model, device, test_loader, criterion)

  scheduler.step()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
            Conv2d-5           [-1, 16, 14, 14]           2,320
       BatchNorm2d-6           [-1, 16, 14, 14]              32
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9             [-1, 32, 7, 7]           9,248
      BatchNorm2d-10             [-1, 32, 7, 7]              64
           Conv2d-11             [-1, 32, 7, 7]           1,056
      BatchNorm2d-12             [-1, 32, 7, 7]              64
          Dropout-13             [-1, 32, 7, 7]               0
AdaptiveAvgPool2d-14             [-1, 3

Train: Loss=0.1463 Batch_id=468 Accuracy=83.06: 100%|██████████| 469/469 [00:35<00:00, 13.21it/s]


Test set: Average loss: 0.0009, Accuracy: 9768/10000 (97.68%)

Epoch 2


Train: Loss=0.2077 Batch_id=468 Accuracy=97.23: 100%|██████████| 469/469 [00:34<00:00, 13.54it/s]


Test set: Average loss: 0.0005, Accuracy: 9832/10000 (98.32%)

Epoch 3


...

Test set: Average loss: 0.0001, Accuracy: 9950/10000 (99.50%)

Epoch 19


Train: Loss=0.0346 Batch_id=468 Accuracy=99.13: 100%|██████████| 469/469 [00:33<00:00, 13.91it/s]


Test set: Average loss: 0.0001, Accuracy: 9943/10000 (99.43%)

Epoch 20


Train: Loss=0.0033 Batch_id=468 Accuracy=99.21: 100%|██████████| 469/469 [00:34<00:00, 13.69it/s]


Test set: Average loss: 0.0001, Accuracy: 9949/10000 (99.49%)

